In [52]:
import os
import numpy as np
import pandas as pd
import xpress as xp
import matplotlib.pyplot as plt

os.environ['XPAUTH_PATH'] = 'xpauth.xpr'

In [87]:
data_path = 'Data/FICO_case_HTW2025'
data_size_list = ['8', '100', '2500']

In [89]:
for data_size in data_size_list:
    print(f"\n=== Processing data_size={data_size} ===")

    df_info = pd.read_csv(f'{data_path}/data{data_size}/shares{data_size}.csv', index_col='Stock')
    df_price = pd.read_csv(f'{data_path}/data{data_size}/stockprices{data_size}.csv', index_col='Stock')

    df_info = df_info.loc[:, ~df_info.columns.str.contains('^Unnamed')]
    df_price = df_price.loc[:, ~df_price.columns.str.contains('^Unnamed')]
    df_price = df_price.astype(float)

    # Remove missing ESG
    missing_data_stocks = df_info[df_info.isna().any(axis=1)].index
    df_info = df_info.drop(missing_data_stocks)
    df_price = df_price.drop(missing_data_stocks)

    # Remove outliers (Z-score)
    df_info_numeric = df_info.select_dtypes(include=[np.number]).columns
    df_info_z = (df_info[df_info_numeric] - df_info[df_info_numeric].mean()) / df_info[df_info_numeric].std(ddof=0)
    df_info_clean = df_info[(np.abs(df_info_z) < 5).all(axis=1)]

    # Remove duplicates
    df_info_clean = df_info_clean[~df_info_clean.index.duplicated(keep='first')]
    df_price_clean = df_price[~df_price.index.duplicated(keep='first')]

    # Remove stocks with constant block
    block_length = 63
    prices_num = df_price_clean.select_dtypes(include=[np.number])
    def_max_run = lambda r: np.diff(np.where(np.concatenate(([True], r.to_numpy()[1:] != r.to_numpy()[:-1], [True])))[0]).max()
    max_runs = prices_num.apply(def_max_run, axis=1)
    valid_stocks = max_runs.index[max_runs < block_length]

    # Ensure same index (only stocks in both datasets & valid)
    final_stocks = df_info_clean.index.intersection(df_price_clean.index).intersection(valid_stocks)
    df_info_clean = df_info_clean.loc[final_stocks]
    df_price_clean = df_price_clean.loc[final_stocks]

    # Save results
    df_info_clean.to_csv(f'{data_path}/data{data_size}/shares{data_size}_cleaned.csv', index=True, index_label='Stock')
    df_price_clean.to_csv(f'{data_path}/data{data_size}/stockprices{data_size}_cleaned.csv', index=True, index_label='Stock')

    # Print summary
    print('df_info original:', df_info.T.shape)
    print('df_info cleaned:', df_info_clean.T.shape)
    print('df_price original:', df_price.T.shape)
    print('df_price cleaned:', df_price_clean.T.shape)



=== Processing data_size=8 ===
df_info original: (3, 8)
df_info cleaned: (3, 8)
df_price original: (5000, 8)
df_price cleaned: (5000, 8)

=== Processing data_size=100 ===
df_info original: (3, 98)
df_info cleaned: (3, 96)
df_price original: (5000, 98)
df_price cleaned: (5000, 96)

=== Processing data_size=2500 ===
df_info original: (3, 2451)
df_info cleaned: (3, 2419)
df_price original: (5000, 2483)
df_price cleaned: (5000, 2419)
